In [12]:
%pip install --upgrade tweepy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import tweepy
import yfinance as yf
import pandas as pd

# Bearer Toke: AAAAAAAAAAAAAAAAAAAAAIXLuAEAAAAAo1Vd4MOQVw7MHSE0GVj8alZjggo%3DUaDUUtafhq9ufMrnHcvO5QsSRj29JzydHBghNZUiNZKpnfQW5d
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAIXLuAEAAAAAo1Vd4MOQVw7MHSE0GVj8alZjggo%3DUaDUUtafhq9ufMrnHcvO5QsSRj29JzydHBghNZUiNZKpnfQW5d'

# Initialize the client
client = tweepy.Client(bearer_token=bearer_token)

# Twitter API credentials (you need to get these from your Twitter Developer account)
consumer_key = 'h345GBAf4Dx8ctWfZPTg4NHm6'
consumer_secret = 'gCYtUAdV7fejFQzXO7MKN1qH5QdhhiHmsDmF4YSTxw9SUQxNLc'
access_token = '1240192427169808385-0QrbbLB1fD8AwzSlEJhva8qwnowvXr'
access_token_secret = 'LtTzs8OvbVimbmtH9LPAhutn8cRGLODJZygMPBL0y2BOU'

# Set up Twitter API client
auth = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret, 
    access_token, access_token_secret
)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [4]:
# Query parameters
query = 'AAPL -is:retweet lang:en'
start_time = '2022-01-01T00:00:00Z'
end_time = '2023-01-01T00:00:00Z'
max_results = 100  # Max results per request (up to 100)

# Collect tweets
tweets = []
for response in tweepy.Paginator(client.search_all_tweets,
                                 query=query,
                                 start_time=start_time,
                                 end_time=end_time,
                                 max_results=max_results,
                                 tweet_fields=['created_at', 'id', 'text', 'author_id'],
                                 limit=10):  # Limit number of requests
    for tweet in response.data:
        tweets.append([tweet.created_at, tweet.id, tweet.text, tweet.author_id])

# Save tweets to DataFrame
tweets_df = pd.DataFrame(tweets, columns=['Date', 'Tweet Id', 'Content', 'Author Id'])

# Display the first 5 tweets
print(tweets_df.head())


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [10]:
# Collect tweets related to a specific stock (e.g., Apple)
tweets = []
query = 'AAPL since:2022-01-01 until:2023-01-01'
for tweet in tweepy.Cursor(api.search_tweets, q=query, lang='en').items(1000):
    tweets.append(tweet.text)

# Save tweets to DataFrame
tweets_df = pd.DataFrame(tweets, columns=['Tweet'])


C:\Users\saad2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tweepy\api.py:301: SyntaxWarning: invalid escape sequence '\['
  """home_timeline(*, count, since_id, max_id, trim_user, \
C:\Users\saad2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tweepy\api.py:341: SyntaxWarning: invalid escape sequence '\['
  """mentions_timeline(*, count, since_id, max_id, trim_user, \
C:\Users\saad2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tweepy\api.py:376: SyntaxWarning: invalid escape sequence '\['
  """user_timeline(*, user_id, screen_name, since_id, count, max_id, \
C:\Users\saad2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tweepy\api.py:426: SyntaxWarning: invalid escape sequence '\

TweepyException: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)

In [8]:
# Collect tweets related to a specific stock (e.g., Apple)
tweets = []
query = "'Elon Musk''fired'-filter:retweets AND -filter:replies AND -filter:links"
# query = 'AAPL since:2022-01-01 until:2023-01-01'
no_of_tweets = 100

try:
    tweets = api.search_tweets(q=query, lang="en", count=no_of_tweets, tweet_mode="extended")

    attributes_container = [[tweet.user.name, tweet.created_at, tweet.favorite_count, tweet.source, tweet.full_text] for tweet in tweets]

    columns = ["User", "Date Created", "Number of Likes", "SOurce of Tweet", "Tweet"]

    tweets_df = pd.DataFrame(attributes_container, columns=columns)

except BaseException as e:
    print("Status Failed on,", str(e) )
# for tweet in tweepy.Cursor(api.search_tweets, q=query, lang='en', count=100).items(1000):
#     tweets.append(tweet.text)


Status Failed on, 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product


In [8]:
import time
from requests.exceptions import Timeout

# Collect tweets related to a specific stock (e.g., Apple)
tweets = []
query = "'Elon Musk' 'fired' -filter:retweets AND -filter:replies AND -filter:links"
no_of_tweets = 100

def fetch_tweets(query, count, max_retries=3):
    for attempt in range(max_retries):
        print(f"Attempt {attempt + 1}")
        try:
            return api.search_tweets(q=query, lang="en", count=count, tweet_mode="extended")
        except (Timeout) as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            time.sleep(5)  # Wait for 5 seconds before retrying
            if attempt == max_retries - 1:
                raise

try:
    tweets = fetch_tweets(query, no_of_tweets)
    attributes_container = [[tweet.user.name, tweet.created_at, tweet.favorite_count, tweet.source, tweet.full_text] for tweet in tweets]
    columns = ["User", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"]
    tweets_df = pd.DataFrame(attributes_container, columns=columns)
except BaseException as e:
    print("Status Failed on,", str(e))

Attempt 1
Status Failed on, Failed to send request: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


------

In [9]:
# Save tweets to DataFrame
tweets_df = pd.DataFrame(tweets, columns=['Tweet'])
tweets_df

,Tweet


In [4]:
# Collect historical stock price data for Apple (AAPL)
stock_df = yf.download('AAPL', start='2022-01-01', end='2023-01-01')
stock_df.reset_index(inplace=True)

KeyboardInterrupt: 

In [2]:
# Save tweets to DataFrame
import pandas as pd
stocks_tweets = pd.read_csv("stock_data.csv")
tweets_df = stocks_tweets[['Text']].copy()  # Select the Text column and make a copy
display(tweets_df)
tweets_df.rename(columns={'Text': 'Tweet'}, inplace=True)  # Rename the column to Tweets
display(tweets_df)

,Text
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...
1,user: AAP MOVIE. 55% return for the FEA/GEED i...
2,user I'd be afraid to short AMZN - they are lo...
3,MNTA Over 12.00
4,OI Over 21.37
...,...
5786,Industry body CII said #discoms are likely to ...
5787,"#Gold prices slip below Rs 46,000 as #investor..."
5788,Workers at Bajaj Auto have agreed to a 10% wag...
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6..."


,Tweet
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...
1,user: AAP MOVIE. 55% return for the FEA/GEED i...
2,user I'd be afraid to short AMZN - they are lo...
3,MNTA Over 12.00
4,OI Over 21.37
...,...
5786,Industry body CII said #discoms are likely to ...
5787,"#Gold prices slip below Rs 46,000 as #investor..."
5788,Workers at Bajaj Auto have agreed to a 10% wag...
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6..."


In [3]:
tweets_df["Tweet"]

0       Kickers on my watchlist XIDE TIT SOQ PNK CPW B...
1       user: AAP MOVIE. 55% return for the FEA/GEED i...
2       user I'd be afraid to short AMZN - they are lo...
3                                       MNTA Over 12.00  
4                                        OI  Over 21.37  
                              ...                        
5786    Industry body CII said #discoms are likely to ...
5787    #Gold prices slip below Rs 46,000 as #investor...
5788    Workers at Bajaj Auto have agreed to a 10% wag...
5789    #Sharemarket LIVE: Sensex off day’s high, up 6...
5790    #Sensex, #Nifty climb off day's highs, still u...
Name: Tweet, Length: 5791, dtype: object

In [22]:
%pip install textblob

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
from textblob import TextBlob
import re

# Function to clean tweets
def clean_tweet(tweet):
    tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)  # Remove mentions
    tweet = re.sub(r'#', '', tweet)  # Remove hashtags
    tweet = re.sub(r'RT[\s]+', '', tweet)  # Remove RT
    tweet = re.sub(r'https?://\S+', '', tweet)  # Remove URLs
    return tweet

# Clean tweets
tweets_df['Tweet'] = tweets_df['Tweet'].apply(clean_tweet)

display(tweets_df)


,Tweet
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...
1,user: AAP MOVIE. 55% return for the FEA/GEED i...
2,user I'd be afraid to short AMZN - they are lo...
3,MNTA Over 12.00
4,OI Over 21.37
...,...
5786,Industry body CII said discoms are likely to s...
5787,"Gold prices slip below Rs 46,000 as investors ..."
5788,Workers at Bajaj Auto have agreed to a 10% wag...
5789,"Sharemarket LIVE: Sensex off day’s high, up 60..."


In [5]:
# Function to get the sentiment of a tweet
def get_sentiment(tweet):
    analysis = TextBlob(tweet)
    return analysis.sentiment.polarity

# Get sentiment scores
tweets_df['Sentiment'] = tweets_df['Tweet'].apply(get_sentiment)

display(tweets_df)

,Tweet,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,0.000000
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1.000000
2,user I'd be afraid to short AMZN - they are lo...,-0.300000
3,MNTA Over 12.00,0.000000
4,OI Over 21.37,0.000000
...,...,...
5786,Industry body CII said discoms are likely to s...,0.000000
5787,"Gold prices slip below Rs 46,000 as investors ...",0.000000
5788,Workers at Bajaj Auto have agreed to a 10% wag...,0.000000
5789,"Sharemarket LIVE: Sensex off day’s high, up 60...",0.098788


In [6]:
# Aggregate sentiment scores by date
tweets_df['Date'] = pd.to_datetime(tweets_df['Tweet'].apply(lambda x: x.created_at.date()))
sentiment_df = tweets_df.groupby('Date')['Sentiment'].mean().reset_index()

# Merge sentiment scores with stock price data
df = pd.merge(stock_df, sentiment_df, on='Date')

display(tweets_df)
display(sentiment_df)
display(df)

AttributeError: 'str' object has no attribute 'created_at'